In [1]:
%cd ..

/media/atem/Data/HSE_videos/Project/sheaf-convolution


/media/atem/Data/HSE_videos/Project/sheaf-convolution/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from utils_comet import export_synth_exp_group_table

PROJECT_NAME = "sheaf-diffusion"
WORKSPACE_NAME = "artem-d"

In [6]:
df_synth = export_synth_exp_group_table(WORKSPACE_NAME, PROJECT_NAME, "size_effect")

In [4]:
df_synth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   model_name            6 non-null      object 
 1   model/total_params    6 non-null      int64  
 2   best_epoch            6 non-null      int64  
 3   best_val_ap           6 non-null      float64
 4   best_val_auc          6 non-null      float64
 5   test_ap_at_best_val   6 non-null      float64
 6   test_auc_at_best_val  6 non-null      float64
 7   dataset/name          6 non-null      object 
 8   dataset/num_nodes     6 non-null      int64  
 9   dataset/num_edges     6 non-null      int64  
 10  dataset/num_features  6 non-null      int64  
 11  dataset/nproj         6 non-null      int64  
dtypes: float64(4), int64(6), object(2)
memory usage: 704.0+ bytes


In [7]:
df_synth

,model_name,model/total_params,best_epoch,best_val_ap,best_val_auc,test_ap_at_best_val,test_auc_at_best_val,dataset/name,dataset/num_nodes,dataset/num_edges,dataset/num_features,dataset/nproj
0,GAE,17728,88,0.613136,0.647002,0.649060,0.664304,SyntheticDataset,200,276,16,4
1,GAE,17728,34,0.623713,0.660799,0.607687,0.630277,SyntheticDataset,200,276,16,4
2,GAE,17728,34,0.589665,0.628137,0.596741,0.624698,SyntheticDataset,200,276,16,4
3,GAE,17728,38,0.590957,0.611688,0.584567,0.605300,SyntheticDataset,1000,6916,16,4
4,GAE,17728,35,0.606413,0.619223,0.572945,0.593499,SyntheticDataset,1000,6916,16,4
5,GAE,17728,95,0.622214,0.649845,0.617870,0.641575,SyntheticDataset,1000,6916,16,4
6,GAE,17728,28,0.619118,0.637422,0.570843,0.580051,SyntheticDataset,2000,27726,16,4
